### Домашнее задание. Реализовать алгоритм Леска и проверить его на реальном датасете

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

**Дополнительный балл**
Если хотите заработать дополнительный балл, попробуйте улучшить алгоритм Леска любым способом (например, использовать расстояние редактирования вместо пересечения или даже вставить машинное обучение)

In [1]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149798 sha256=140adb349b15d0a84d56d9c79ebe55d710529a838d5db9151ea643f5fb5fd971
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-levenshtein


In [2]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import nltk
from fuzzywuzzy import fuzz
from string import punctuation
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
def fuzzy_compare(sent1: str, sent2: str) -> int:
    """ one of the options to compare an example and a definition in Lesk algo"""
    return fuzz.token_sort_ratio(sent1, sent2)

def lesk(word: str, sentence: str, compare_how: callable) -> int:
    """ 
        disambiguate a given word using nltk.wordnet
        return an index of a word the word's lemma is not known.
        Pass a part of speech first
    """
    bestsense = 0
    maxoverlap = 0
    word=wn.morphy(word) if wn.morphy(word) is not None else word
    synsets = wn.synsets(word)
 
    for i, syns in enumerate(synsets):
        overlap = compare_how(sentence, syns.definition())
                
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
    return bestsense

In [5]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time', compare_how=fuzzy_compare) # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

**Проверьте насколько хорошо работает такой метод на реальном датасете.** http://lcl.uniroma1.it/wsdeval/ - большой фреймворк для оценки WSD. Там много данных и я взял кусочек, чтобы не было проблем с памятью

Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [6]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [15]:
true = 0
total = 0

for sent in corpus_wsd[0:1000]:
    for word in sent:
        meaning, lemma, word = word
        if not meaning:
            continue
        total += 1
        context = " ".join([s[2] for s in sent])
        meaning_id = lesk(word, context, compare_how=fuzzy_compare)
        try:
            if wn.synsets(lemma)[meaning_id] == wn.lemma_from_key(meaning).synset():
                true += 1
        except:
            # somehow, several meaning_ids are out of range
            pass

print('percentage of correct: %f' % (round((true/total) * 100, 1)))

percentage of correct: 37.600000
